# 02-Projeto 1: Tratamento, Exploração e Análise de Dados

Este projeto analisa os dados dos casos de covid 19 no 
estado de São Paulo do período de fevereiro de 2020 a fevereiro de 2021

## Importação dos Dados

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
covid_sp = pd.read_csv('./dados_covid_sp.csv', sep=';', encoding='utf-8')
# sep -> tipo de separador dos dados
# encoding -> codificação dos caracteres: utf-8, iso-8859-1, latin-1, etc

In [ ]:
covid_sp.head()

In [ ]:
covid_sp.shape

## Organização dos Dados

### Renomeando variáveis (colunas)

In [ ]:
covid_sp = covid_sp.rename(columns={'nome_munic': 'municipio'})

In [ ]:
covid_sp.head(3)

In [ ]:
covid_sp.rename(columns={'datahora': 'data'}, inplace = True) 
# inplace = True confirma a alteração na tabela

In [ ]:
covid_sp.rename(columns={'map_leg': 'rotulo_mapa', 'map_leg_s': 'codigo_mapa'}, inplace = True) 

### Excluindo variáveis (colunas)

In [ ]:
covid_sp_alterado = covid_sp.drop(columns=['cod_ra'])
# realizar a alteração e atribuir a uma nova tabela

In [ ]:
covid_sp_alterado.head(1)

In [ ]:
covid_sp_alterado = covid_sp_alterado.drop(covid_sp_alterado.columns[[1]], axis=1)

In [ ]:
covid_sp_alterado.drop(columns=['rotulo_mapa', 'codigo_mapa', 'cod_drs'], inplace = True) 

In [ ]:
covid_sp_alterado.drop(covid_sp_alterado.columns[[13,14,18,19]],axis=1, inplace=True)

In [ ]:
covid_sp_alterado.shape

### Criando e Alterando valores das colunas (variáveis)

In [ ]:
covid_sp_alterado.head(2)

In [ ]:
# corrigindo o valor de área
covid_sp_alterado['area'] = covid_sp_alterado['area']/100

In [ ]:
# criação de uma coluna com a densidade demográfica (hab/km2)

covid_sp_alterado['densidade'] = covid_sp_alterado['pop'] / covid_sp_alterado['area']

In [ ]:
covid_sp_alterado.shape

In [ ]:
# criação de uma coluna com índices

index = list(range(1, 374035))

In [ ]:
# transformar a lista em um dataframe

df = pd.DataFrame(index,columns=['indice'])
df

In [ ]:
# juntar os dois dataframes

covid_sp_alterado = pd.concat([covid_sp_alterado, df],axis=1)
# axis = 1 -> coluna
# axis = 0 -> linha

In [ ]:
# reorganizar a tabela

covid_sp_alterado = covid_sp_alterado.reindex(columns=['indice'] + list(covid_sp_alterado.columns[:-1]))

### Contagem de Registros das Variáveis (colunas)

In [ ]:
# contagem de dados

covid_sp_alterado['semana_epidem'].value_counts()

In [ ]:
covid_sp_alterado['semana_epidem'].value_counts().sort_index()

In [ ]:
from collections import Counter


Counter(covid_sp_alterado.semana_epidem)

In [ ]:
# realizando uma contagem por municipios

covid_sp_alterado.query('obitos_novos > 50')['municipio'].value_counts()

### Selecionar Variáveis (colunas) por índices

In [ ]:
x = covid_sp_alterado.iloc[0:10 , 5:13] # iloc[linhas, colunas]
x

In [ ]:
# extrair os valores das linhas
# selecionar todas as linhas da coluna 1

y = covid_sp_alterado.iloc[:,1].values

In [ ]:
# agora transformar esses valores em uma lista

lista_y = list(y.flatten())

In [ ]:
df = pd.DataFrame(lista_y, columns=['municipio'])
df

### Excluindo, filtrando e substituindo registros (linhas)

In [ ]:
# excluindo linhas por índices (valores absolutos)
covid_sp_alterado2 = covid_sp_alterado.drop(covid_sp_alterado.index[[1,3]])

In [ ]:
covid_sp_alterado2.head(5)

In [ ]:
# excluindo linhas por índices (intervalos de valores)
covid_sp_alterado2 = covid_sp_alterado.drop(covid_sp_alterado.index[4:7])

In [ ]:
# reordenar (resetar) os índices
covid_sp_alterado2 = covid_sp_alterado2.reset_index(drop=True) # drop é para excluir o índice anterior

In [ ]:
# encontrando municípios com o nome 'ignorado'

ignorado = covid_sp_alterado.loc[covid_sp_alterado.municipio =='Ignorado']
ignorado

In [ ]:
ignorado.shape

In [ ]:
# filtrando a tabela para ignorar os valores de municipio = Ignorado
covid_sp_alterado = covid_sp_alterado.loc[covid_sp_alterado.municipio != 'Ignorado']
covid_sp_alterado

In [ ]:
# casos de covid da cidade de guarulhos

guarulhos = covid_sp_alterado.loc[covid_sp_alterado.municipio == 'Guarulhos']
guarulhos

In [ ]:
# remover a coluna data e municipio de Guarulhos

guarulhos.drop(columns=['data','municipio'], inplace=True)

In [ ]:
guarulhos.head(5)

In [ ]:
# substituir valores da tabela guarulhos

guarulhos['semana_epidem']=guarulhos['semana_epidem'].replace({9:'nove', 10:'dez'})

In [ ]:
# outra maneira de substituir valores

guarulhos['semana_epidem']=guarulhos['semana_epidem'].replace([11,12,13],['onze','doze','treze'])

In [ ]:
# substituir ',' por '.'

guarulhos['casos_pc'] = guarulhos['casos_pc'].apply(lambda x: x.replace(',','.'))

In [ ]:
# criar uma coluna com as datas

import datetime

In [ ]:
data = np.array('2020-02-25', dtype = np.datetime64())
data

In [ ]:
data = data + np.arange(579)
data

In [ ]:
data = pd.DataFrame(data)
data

In [ ]:
data.columns=['data']
data.head()

In [ ]:
guarulhos2 = pd.concat([data,guarulhos],axis=1)
guarulhos2.head()

In [ ]:
# reordenar os índices

guarulhos = guarulhos.reset_index(drop=True)

In [ ]:
guarulhos2 = pd.concat([data,guarulhos],axis=1)
guarulhos2.head()

In [ ]:
guarulhos2.shape

### Valores Missing

In [ ]:
# Relação da Quantidade

covid_sp_alterado.isnull().sum()

In [ ]:
# Analisando apenas uma coluna

covid_sp_alterado['casos'].isnull().sum()

In [ ]:
# Relação da Quantidade na tabela original

covid_sp.isnull().sum()

In [ ]:
# Excluindo os dados nulos

covid_sp2 = covid_sp.dropna()

In [ ]:
# Preencher o valores Missing com mediana

covid_sp['obitos_novos'].fillna(covid_sp['obitos_novos'].median(), inplace=True)

In [ ]:
# Preencher o valores Missing com a media

covid_sp['obitos_novos'].fillna(covid_sp['obitos_novos'].mean(), inplace=True)

In [ ]:
# Preencher o valores Missing com um valor qualquer

covid_sp['obitos_novos'].fillna(10, inplace=True)

### Classificação e alteração da tipagem dos atributos

In [ ]:
# identificação do tipo de dados

covid_sp_alterado.dtypes

In [ ]:
# alterando o tipo de dado de casos_pc para float

covid_sp_alterado['casos_pc'] = covid_sp_alterado['casos_pc'].astype(float)

In [ ]:
# antes devemos alterar ',' para '.' em casos_pc

covid_sp_alterado['casos_pc'] = covid_sp_alterado['casos_pc'].apply(lambda x: x.replace(',','.'))

In [ ]:
covid_sp_alterado.head(2)

In [ ]:
# fazer o mesmo tratamento para todas variáveis com ',' para '.'

covid_sp_alterado['casos_mm7d'] = covid_sp_alterado['casos_mm7d'].apply(lambda x: x.replace(',','.'))
covid_sp_alterado['obitos_pc'] = covid_sp_alterado['obitos_pc'].apply(lambda x: x.replace(',','.'))
covid_sp_alterado['obitos_mm7d'] = covid_sp_alterado['obitos_mm7d'].apply(lambda x: x.replace(',','.'))
covid_sp_alterado['letalidade'] = covid_sp_alterado['letalidade'].apply(lambda x: x.replace(',','.'))

In [ ]:
# agora transformar os tipos dos dados 

covid_sp_alterado['casos_mm7d'] = covid_sp_alterado['casos_mm7d'].astype(float)
covid_sp_alterado['obitos_pc'] = covid_sp_alterado['obitos_pc'].astype(float)
covid_sp_alterado['obitos_mm7d'] = covid_sp_alterado['obitos_mm7d'].astype(float)
covid_sp_alterado['letalidade'] = covid_sp_alterado['letalidade'].astype(float)

In [ ]:
# alterar o tipo de data também

covid_sp_alterado['data'] = covid_sp_alterado['data'].astype('datetime64[s]')

## Salvando (Exportando) o DataFrame Tratado

In [ ]:
covid_sp_alterado.to_csv('covid_sp_tratado.csv', sep=';', encoding='utf-8', index=False)